In [ ]:
# Import libraries
import pandas as pd
import json
import re

# Load JSON files
file_paths = [
    './01_Posts_HiddenAnswers_ptbr_01.json',
    './02_Posts_HiddenAnswers_ptbr_02.json',    
    './03_Posts_DeepAnswers_ptbr_01.json'
    
]

dataframes = []

for file_path in file_paths:
    with open(file_path, 'r') as file:
        data = json.load(file)
        df = pd.json_normalize(data)
        
                
        # Renaming columns 'question' to 'content' and 'dataCreated' to 'created_at'
        df = df.rename(columns={'question': 'content', 'dataCreated': 'created_at'})
        
        dataframes.append(df)


# Merge dataframes
df = pd.concat(dataframes, ignore_index=True)


# Patterns for date columns
date_columns_patterns = [
    ('created_at', r'^created_at$')
]

# Convert date columns to datetime without time zone
for column_name, pattern in date_columns_patterns:
    matching_columns = [col for col in df.columns if re.match(pattern, col, re.I)]
    for col in matching_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce').dt.tz_localize(None)

# Selecting columns of interest
columns_of_interest = ['category', 'title', 'content', 'answers', 'tags', 'created_at']
df = df[columns_of_interest]

# Add 'id' column
df['id'] = range(1, len(df) + 1)

# Reorganize columns
df = df[['id'] + [col for col in df.columns if col != 'id']]


# Create a 'full_text_ioc' column by concatenating the 'title', 'content', 'answers', and 'tags' columns
df['full_text_ioc'] = df['title'].astype(str) + ' ' + df['content'].astype(str) + ' ' + df['answers'].astype(str) + ' ' + df['tags'].astype(str)


# Preprocessing function (removes irrelevant information)
def preprocess_text(text):
    patterns_to_remove = [
        r"'title': [^,]*,",
        r"'name': [^,]*,",
        r"'type': [^,]*,",
        r"'author': [^,]*,",
        r"'id': [^,]*,",
        r"'questionid': [^,]*,",
        r"'answerid': [^,]*,"
    ]
    for pattern in patterns_to_remove:
        text = re.sub(pattern, ' ', text)
    return text

# Apply the preprocessing function
df['full_text_ioc'] = df['full_text_ioc'].apply(preprocess_text)


# Save CSV file
df.to_csv('df_pre_process_I.csv', index=False)

# Save XLSX file
df.to_excel('df_pre_process_I.xlsx', index=False)

In [ ]:
#df